# ML-обработка результатов голосований Госдумы 7-го созыва (2016-2021)

### Парсер результатов голосования с сайта http://vote.duma.gov.ru/ . Были собраны все данные о голосованиях госдумы 7-го созыва (2016-2021 г.)

In [1]:
import requests
import pandas as pd
import json
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

In [2]:
# Диапазон номеров голосований
# 96006 - номер первого голосования госдумы 7-го созыва
# 114772 - номер последнего голосования на момент сбора датасета (04.06.2021)
page_list = list(range(114772, 96006, -1))

In [3]:
len(page_list)

18766

In [4]:
unparsed = []
# Словарь с результатами голосований
votes = {}
# Словарь с партиной принадлежностью депутатов
dep_part_dict = {}
for page in tqdm(page_list):
    page_link = "http://vote.duma.gov.ru/vote/" + str(page)
    try:
        response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
        html = response.content
        soup = BeautifulSoup(html, 'lxml')
        temp_str = str(soup.findAll('script')[10])
        str_pos = temp_str.find('deputiesData ')
        temp_str2 = temp_str[str_pos + 15 : temp_str.find(']', str_pos)+1]
        dip_list = json.loads(temp_str2)
        dep_dict = {}
        for d in dip_list:
            dep_dict[d['sortName']] = d['result']
            if d['sortName'] not in dep_part_dict:
                dep_part_dict[d['sortName']] = d['faction']

        law_name = soup.find('div', attrs = {'class':'box-p'}).h1.text
        vote_result = soup.find('div', attrs = {'class':'statis-p'}).b.text.split()[0]
        try:
            votes_yes = soup.find('div', attrs = {'class':'statis-p'}).b.text.split()[1][1:]
            t = soup.find('div', attrs = {'class':'statis-p'}).b.text.split()[3]
            votes_yes_perc = t[:t.find('%')]
        except:
            votes_yes = None
            votes_yes_perc = None
        votes[page] = [law_name, vote_result, votes_yes, votes_yes_perc, dep_dict]
    except:
        #print(f"error, unparsed = {page}")
        unparsed.append(page)
        continue
    
    #time.sleep(1 + np.random.rand())

In [5]:
len(votes)

16249

In [34]:
# Как выглядят данные по первому сохранённому голосованию
votes[next(iter(votes))]

['(за основу) О проекте порядка работы Государственной Думы на 2 июня 2021 года',
 'принят',
 '383',
 '85.1',
 {'Авдеев Александр Александрович': -1,
  'Адучиев Батор Канурович': -1,
  'Азимов Рахим Азизбоевич': -1,
  'Аксаков Анатолий Геннадьевич': 2,
  'Алексеева Татьяна Олеговна': -1,
  'Алимова Ольга Николаевна': -1,
  'Альшевских Андрей Геннадьевич': -1,
  'Ананских Игорь Александрович': 2,
  'Андрейченко Андрей Валерьевич': -1,
  'Аникеев Григорий Викторович': -1,
  'Антонова Лидия Николаевна': -1,
  'Арефьев Николай Васильевич': -1,
  'Аршба Отари Ионович': -1,
  'Аршинова Алёна Игоревна': -1,
  'Аскендеров Заур Асевович': -1,
  'Афонин Юрий Вячеславович': -1,
  'Афонский Владимир Игорьевич': -1,
  'Байгускаров Зариф Закирович': -1,
  'Балыбердин Алексей Владимирович': -1,
  'Балыхин Григорий Артёмович': -1,
  'Бальбек Руслан Исмаилович': -1,
  'Бариев Марат Мансурович': -1,
  'Барышев Андрей Викторович': 2,
  'Баталова Рима Акбердиновна': -1,
  'Бахарев Константин Михайлович': 

In [8]:
# Построение таблицы с результатами голосований
dep_votes = {}
for v in votes:
    for d in votes[v][4]:
        if d in dep_votes:
            dep_votes[d][v] = votes[v][4][d]
        else:
            dep_votes[d] = {v: votes[v][4][d]}

In [9]:
len(dep_votes)

499

Сборка датафрейма с результатами голосований

In [10]:
df = pd.DataFrame(index=[v for v in votes])
df.head()

""
114772
114771
114770
114769
114768


In [11]:
df1 = pd.DataFrame([votes[v][0] for v in votes], index=[v for v in votes], columns=['law_name'])
df2 = pd.DataFrame([votes[v][1] for v in votes], index=[v for v in votes], columns=['vote_result'])

In [12]:
for d in dep_votes:
    df = df.join(pd.DataFrame([dep_votes[d][v] for v in dep_votes[d]], index=[v for v in dep_votes[d]], columns=[d]))

In [37]:
df.iloc[:3, :10]

,Авдеев Александр Александрович,Адучиев Батор Канурович,Азимов Рахим Азизбоевич,Аксаков Анатолий Геннадьевич,Алексеева Татьяна Олеговна,Алимова Ольга Николаевна,Альшевских Андрей Геннадьевич,Ананских Игорь Александрович,Андрейченко Андрей Валерьевич,Аникеев Григорий Викторович
114772,-1.0,-1.0,-1.0,2.0,-1.0,-1.0,-1.0,2.0,-1.0,-1.0
114771,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
114770,2.0,2.0,2.0,2.0,2.0,-1.0,2.0,2.0,-1.0,2.0


In [14]:
df_full = df1.join(df2).join(df)

In [38]:
df_full.iloc[:3, :10]

,law_name,vote_result,Авдеев Александр Александрович,Адучиев Батор Канурович,Азимов Рахим Азизбоевич,Аксаков Анатолий Геннадьевич,Алексеева Татьяна Олеговна,Алимова Ольга Николаевна,Альшевских Андрей Геннадьевич,Ананских Игорь Александрович
114772,(за основу) О проекте порядка работы Государст...,принят,-1.0,-1.0,-1.0,2.0,-1.0,-1.0,-1.0,2.0
114771,Регистрация,принят,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
114770,(первое чтение) О проекте федерального закона ...,отклонен,2.0,2.0,2.0,2.0,2.0,-1.0,2.0,2.0


In [16]:
#df_full.to_csv('./data/df_full.csv')

In [22]:
#df_full = pd.read_csv('./data/df_full.csv', index_col=0)

In [17]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16249 entries, 114772 to 96007
Columns: 501 entries, law_name to Нарышкин Сергей Евгеньевич
dtypes: float64(499), object(2)
memory usage: 62.9+ MB


In [64]:
df_full_cr = df_full.copy()

In [65]:
# Доля пропущенных голосований
df_full_cr.isnull().sum() / len(votes)

law_name                            0.000000
vote_result                         0.000000
Авдеев Александр Александрович      0.000185
Адучиев Батор Канурович             0.000185
Азимов Рахим Азизбоевич             0.000185
                                      ...   
Любимов Николай Викторович          0.910394
Меткин Александр Михайлович         0.910394
Скоробогатько Александр Иванович    0.967321
Макаровец Николай Александрович     0.993169
Нарышкин Сергей Евгеньевич          0.993169
Length: 501, dtype: float64

In [68]:
# Депутаты, которые пропустили более 10% голосований
t = (df_full_cr.isnull().sum() / len(votes)) > 0.1
t

law_name                            False
vote_result                         False
Авдеев Александр Александрович      False
Адучиев Батор Канурович             False
Азимов Рахим Азизбоевич             False
                                    ...  
Любимов Николай Викторович           True
Меткин Александр Михайлович          True
Скоробогатько Александр Иванович     True
Макаровец Николай Александрович      True
Нарышкин Сергей Евгеньевич           True
Length: 501, dtype: bool

In [69]:
sum(t[t == True])

93

In [70]:
# Удаляем депутатов с большим количеством пропусков (т.е. которые не были в составе думы в течение всего созыва)
df_full_cr = df_full_cr.drop(columns=(t[t == True].index))

In [71]:
# Для остальных заполняем пропуски значением 2 (не голосовал)
df_full_cr = df_full_cr.fillna(2)

In [72]:
df_full_cr.iloc[:3, :10]

,law_name,vote_result,Авдеев Александр Александрович,Адучиев Батор Канурович,Азимов Рахим Азизбоевич,Аксаков Анатолий Геннадьевич,Алексеева Татьяна Олеговна,Альшевских Андрей Геннадьевич,Ананских Игорь Александрович,Аникеев Григорий Викторович
114772,(за основу) О проекте порядка работы Государст...,принят,-1.0,-1.0,-1.0,2.0,-1.0,-1.0,2.0,-1.0
114771,Регистрация,принят,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
114770,(первое чтение) О проекте федерального закона ...,отклонен,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0


In [78]:
# Для удобства превращаем список депутатов в pd.Series
dep_df = pd.Series(dep_part_dict)
dep_df.head(3)

Авдеев Александр Александрович    ЕР
Адучиев Батор Канурович           ЕР
Азимов Рахим Азизбоевич           ЕР
dtype: object

In [79]:
# Исключаем из списка тех депутатов, которые были удалены из основного датафрейма, и сохраняем
dep_df_cr = dep_df.loc[df_full_cr.columns[2:].values]

dep_df_cr.to_csv('./data/dep_df_cr.csv')

In [73]:
df_full_cr_0 = df_full_cr.copy()

# Преобразование типов в int8
for d in df_full_cr.drop(columns=['law_name', 'vote_result']).columns:
    df_full_cr = df_full_cr.astype({d: 'int8'})
    df_full_cr_0 = df_full_cr_0.astype({d: 'int8'})
    
    # Приравниваем неголосовавших депутатов к воздержавшимся
    df_full_cr_0[d] = df_full_cr_0[d].map(lambda x: 0 if x == 2 else x)

In [74]:
df_full_cr_0.iloc[:3, :10]

,law_name,vote_result,Авдеев Александр Александрович,Адучиев Батор Канурович,Азимов Рахим Азизбоевич,Аксаков Анатолий Геннадьевич,Алексеева Татьяна Олеговна,Альшевских Андрей Геннадьевич,Ананских Игорь Александрович,Аникеев Григорий Викторович
114772,(за основу) О проекте порядка работы Государст...,принят,-1,-1,-1,0,-1,-1,0,-1
114771,Регистрация,принят,0,0,0,0,0,0,0,0
114770,(первое чтение) О проекте федерального закона ...,отклонен,0,0,0,0,0,0,0,0


In [75]:
df_full_cr.shape

(16249, 408)

In [76]:
df_full_cr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16249 entries, 114772 to 96007
Columns: 408 entries, law_name to Шурчанов Валентин Сергеевич
dtypes: int8(406), object(2)
memory usage: 7.3+ MB


In [77]:
df_full_cr.to_csv('./data/df_full_cr.csv')
df_full_cr_0.to_csv('./data/df_full_cr_0.csv')